<a href="https://colab.research.google.com/github/PeroronShine/AndroidApp/blob/main/AI_prog/ML_lab4_accident_risk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pandas numpy scikit-learn tensorflow

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

/tmp/ipython-input-1658607810.py:1: DtypeWarning: Columns (7,8,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv('train.csv')


In [ ]:
print("\nПервые 5 строк train.csv:")
print(train_df.head())
print(test_df.head())


Первые 5 строк train.csv:
   id road_type  num_lanes  curvature  speed_limit  lighting weather  \
0   0     urban          2       0.06           35  daylight   rainy   
1   1     urban          4       0.99           35  daylight   clear   
2   2     rural          4       0.63           70       dim   clear   
3   3   highway          4       0.07           35       dim   rainy   
4   4     rural          1       0.58           60  daylight   foggy   

  road_signs_present public_road time_of_day holiday school_season  \
0              False        True   afternoon   False          True   
1               True       False     evening    True          True   
2              False        True     morning    True         False   
3               True        True     morning   False         False   
4              False       False     evening    True         False   

   num_reported_accidents  accident_risk  
0                     1.0           0.13  
1                     0.0        

In [ ]:
# Разделение признаков и целевой переменной
target_col = 'accident_risk'
feature_cols = [col for col in train_df.columns if col not in ['id', target_col]]

X = train_df[feature_cols]
y = train_df[target_col]

X_test = test_df[feature_cols]
test_ids = test_df['id']

mask = y.notna()
X = X[mask]
y = y[mask]

# Проверка: одинаковые колонки в train и test?
assert set(X.columns) == set(X_test.columns), "Признаки в train и test не совпадают!"


In [ ]:
# Обработка пропусков: заполняем числовые NaN средним, категориальные — модой
for col in X.columns:
    if X[col].dtype == 'object' or X[col].dtype == 'bool':
        mode_val = X[col].mode()[0] if not X[col].mode().empty else 'missing'
        X[col].fillna(mode_val, inplace=True)
        X_test[col].fillna(mode_val, inplace=True)
    else:
        mean_val = X[col].mean()
        X[col].fillna(mean_val, inplace=True)
        X_test[col].fillna(mean_val, inplace=True)

# Обработка категориальных признаков
categorical_cols = X.select_dtypes(include=['object', 'bool']).columns.tolist()

for col in categorical_cols:
    # Приведение к строке, чтобы избежать смешанных типов
    X[col] = X[col].astype(str)
    X_test[col] = X_test[col].astype(str)

    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])
    X_test[col] = le.transform(X_test[col])

/tmp/ipython-input-2600818028.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X[col].fillna(mode_val, inplace=True)
/tmp/ipython-input-2600818028.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.met

In [ ]:
# Нормализация признаков
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_test_scaled = scaler.transform(X_test)

# Разделение на train/val
X_train, X_val, y_train, y_val = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)

In [ ]:
# Построение модели нейронной сети
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(1, activation='sigmoid')  # sigmoid для [0,1]
])

# Компиляция модели
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='mse',  # MSE подходит для регрессии
    metrics=['mae']
)

# Ранняя остановка
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

print("Обучение модели...")
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=100,
    batch_size=32,
    callbacks=[early_stopping],
    verbose=1
)

Обучение модели...
Epoch 1/100


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1813/1813 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 0.0130 - mae: 0.0878 - val_loss: 0.0046 - val_mae: 0.0530
Epoch 2/100
1813/1813 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 0.0056 - mae: 0.0588 - val_loss: 0.0039 - val_mae: 0.0481
Epoch 3/100
1813/1813 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0049 - mae: 0.0544 - val_loss: 0.0037 - val_mae: 0.0473
Epoch 4/100
1813/1813 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0046 - mae: 0.0525 - val_loss: 0.0037 - val_mae: 0.0469
Epoch 5/100
1813/1813 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0044 - mae: 0.0514 - val_loss: 0.0037 - val_mae: 0.0470
Epoch 6/100
1813/1813 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 0.0042 - mae: 0.0506 - val_loss: 0.0037 - val_mae: 0.0470
Epoch 7/100
1813/1813 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0042 - mae: 0.0504 - val_loss: 0.0035 - val_mae: 0.0458
Epoch 8/100
1813/1813 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0042 - mae: 0.0505 - val_loss: 0.0035 - val_mae: 0.0455
Epoch 9/100
1813/1813 ━━━━━━━━━━━━━━━━━━━━ 4

In [ ]:
# Оценка модели на валидации
val_pred = model.predict(X_val).flatten()
mse = mean_squared_error(y_val, val_pred)
mae = mean_absolute_error(y_val, val_pred)
r2 = r2_score(y_val, val_pred)

print("\n📊 Метрики на валидационной выборке:")
print(f"MSE: {mse:.6f}")
print(f"MAE: {mae:.6f}")
print(f"R²: {r2:.6f}")

454/454 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step  

📊 Метрики на валидационной выборке:
MSE: 0.003419
MAE: 0.045157
R²: 0.876225


In [ ]:
y_pred = model.predict(X_test_scaled).flatten()

# Статистика по предсказаниям
print("\n📊 Статистика по предсказаниям для test:")
print(f"Минимум: {y_pred.min():.6f}")
print(f"Максимум: {y_pred.max():.6f}")
print(f"Среднее: {y_pred.mean():.6f}")
print(f"Стандартное отклонение: {y_pred.std():.6f}")

1911/1911 ━━━━━━━━━━━━━━━━━━━━ 1s 705us/step

📊 Статистика по предсказаниям для test:
Минимум: 0.032519
Максимум: 0.955887
Среднее: 0.354134
Стандартное отклонение: 0.157008


In [ ]:
submission = pd.DataFrame({
    'id': test_ids,
    'accident_risk': y_pred
})

# Сохранение
submission.to_csv('submission.csv', index=False)
